In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv'

## Statistical investigation of the data

In [ ]:
#Includes information for all columns (both numerical and categorical).
df.describe(include = "all")

### Filling missing values with median/mean (for proper outlier detection)

In [ ]:
#Information on sections of each column that are null. 
print(df.isnull().sum())

Date                 0
Location             0
MinTemp           1043
MaxTemp            927
Rainfall          2923
Evaporation      62400
Sunshine         69447
WindGustDir       9892
WindGustSpeed     9830
WindDir9am       10111
WindDir3pm        3784
WindSpeed9am      1313
WindSpeed3pm      2618
Humidity9am       2199
Humidity3pm       4065
Pressure9am      14625
Pressure3pm      14588
Cloud9am         55433
Cloud3pm         58903
Temp9am           1312
Temp3pm           3167
RainToday         2923
RainTomorrow      2972
dtype: int64


In [ ]:
#Columns or rows should only be dropped if >70% of it is empty or missing. 
threshold = 0.7

#Dropping columns with missing value rate higher than threshold
df = df[df.columns[df.isnull().mean() < threshold]]

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

df

#### Non-prediction Imputation Methods (this may suggest to us that prediction imputation methods are more sound)

In [ ]:
#Fill missing categorical values with maximum occurred value. 
dfNonPrediction = df.apply(lambda x:x.fillna(x.value_counts().index[0]))

#Version one: fill missing numerical values with median.
dfNonPredictionMedian = dfNonPrediction.fillna(dfNonPrediction.median())

#Version two: fill missing numerical values with mean.
dfNonPredictionMean = dfNonPrediction.fillna(dfNonPrediction.median())


In [ ]:
#Information on sections of each column that are null (dfNonPredictionMedian). 
print(dfNonPredictionMedian.isnull().sum())

#Information on sections of each column that are null (dfNonPredictionMean). 
print(dfNonPredictionMean.isnull().sum())

In [ ]:
dfNonPredictionMean

In [ ]:
dfNonPredictionMedian

### Drop rows with at least one missing value

In [ ]:
#Dropping rows with at least one missing value. The outcome is a dataset with no missing values whatsoever. 
dfNoMissing = df.loc[df.isnull().mean(axis=1) < (1/23)]
dfNoMissing

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,2017-06-20,Darwin,19.3,33.4,0.0,6.0,11.0,ENE,35.0,SE,NE,9.0,20.0,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,No,No
142299,2017-06-21,Darwin,21.2,32.6,0.0,7.6,8.6,E,37.0,SE,SE,13.0,11.0,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,No,No
142300,2017-06-22,Darwin,20.7,32.8,0.0,5.6,11.0,E,33.0,E,W,17.0,11.0,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,No,No
142301,2017-06-23,Darwin,19.5,31.8,0.0,6.2,10.6,ESE,26.0,SE,NNW,9.0,17.0,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,No,No


In [ ]:
print(dfNonPredictionMedian.isnull().sum())
dfNoMissing.hist(figsize=(25, 15))
plt.show()

### Data Visualizations

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets

#Distributions with medians filled in for missing values.
dfNonPredictionMedian.hist(figsize=(25, 15))
plt.show()

In [ ]:
#Distributions with means filled in for missing values.
dfNonPredictionMean.hist(figsize=(25, 15))
plt.show()

In [ ]:
#Distributions on the original/unmodified dataset.
df.hist(figsize=(25, 15))
plt.show()

Based on the distributions above, clearly some attributes have outliers. Below, we will show the records that have such outlier values. The goal is to see whether an attribute's outlier values occurs *across* the record (i.e. if a value is a true outlier, other values in the record or observation are likely to be outliers as well).

In [ ]:
#Check all the observations that contain outliers.
from scipy import stats

numDfNoMissing = dfNoMissing.select_dtypes(include=["number"])
catDfNoMissing = dfNoMissing.select_dtypes(exclude=["number"])

numDfNoMissingNoOutliers = numDfNoMissing[(np.abs(stats.zscore(numDfNoMissing)) < 3).all(axis=1)]

In [ ]:
#Below are the outliers. Out of all the records that are complete, 2730 are outliers. 
pd.concat([numDfNoMissing,numDfNoMissingNoOutliers]).drop_duplicates(keep=False)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
6063,25.2,40.5,0.0,16.4,10.3,44.0,13.0,22.0,24.0,15.0,1012.4,1009.0,4.0,6.0,31.4,37.8
6065,18.4,31.8,0.0,16.0,12.9,33.0,17.0,15.0,25.0,5.0,1016.3,1013.8,0.0,1.0,19.9,30.3
6078,25.1,39.3,0.0,15.8,13.2,44.0,20.0,20.0,34.0,18.0,1013.8,1010.5,0.0,1.0,29.9,37.5
6079,25.2,38.5,0.0,16.2,13.1,44.0,24.0,20.0,35.0,18.0,1015.2,1011.9,1.0,2.0,29.0,36.9
6084,27.2,43.4,0.0,14.2,12.6,37.0,15.0,7.0,33.0,14.0,1005.9,1003.3,1.0,3.0,32.6,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142206,24.1,29.3,22.0,9.4,0.4,31.0,13.0,7.0,91.0,79.0,1011.5,1006.9,8.0,8.0,24.1,28.5
142209,24.9,31.9,20.0,4.0,4.8,28.0,7.0,13.0,88.0,69.0,1010.4,1006.8,7.0,7.0,27.6,30.3
142210,24.5,32.3,18.0,3.2,5.8,31.0,7.0,7.0,82.0,86.0,1009.5,1005.7,5.0,7.0,28.0,27.5
142211,24.0,30.4,17.0,5.8,0.8,31.0,11.0,4.0,89.0,87.0,1007.9,1006.0,7.0,8.0,26.7,27.7


The final analysis is to *not* remove the outliers. The above are true outliers. 

## Data pre-processing

### Modifying Date and Location Attributes (Neither of these contain missing values)

In [ ]:
#Convert date into time of year.
df['Date'] = df['Date'].str[5:]
labelEncoder = preprocessing.LabelEncoder()

#Generate integer encodings because there is a natural ordinal relationship between the values.
df['Date'] = labelEncoder.fit_transform(df['Date'])
print(df['Date'])

0         0
1         0
2         0
3         0
4         0
         ..
145455    0
145456    0
145457    0
145458    0
145459    0
Name: Date, Length: 145460, dtype: int64


In [ ]:
#Convert locations into latitude and longitude coordinates.

#Import and installation statements.
!pip install geopy
from geopy.geocoders import Nominatim
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#Make a list containing unique locations.
uniqueLocations = df['Location'].unique().tolist()
locationsDict = {}

#Format locations correctly for correct input to geopy.
regex = re.compile('(?!^)(?=[A-Z])', re.MULTILINE)

geolocator = Nominatim()

#Add each location with its coordinates to locationsDict.
for locationKey in uniqueLocations:
  locationKeyModified = re.sub(regex, " ", locationKey)
  
  #Change PerthRAAF manually.
  if ("R A A F" in locationKeyModified):
    locationKeyModified = locationKeyModified[:locationKeyModified.find(" ")] + " RAAF"
  location = geolocator.geocode(locationKeyModified + ", Australia")
  locationsDict[locationKey] = (location.latitude, location.longitude)

latitudes = []
longitudes = []  

#Add coordinates to dataset columns.
for index, row in df.iterrows():
  latitudes.append(locationsDict[df['Location'][index]][0])
  longitudes.append(locationsDict[df['Location'][index]][1])

df.insert(1, "Latitude", latitudes) 
df.insert(2, "Longitude", longitudes) 
df = df.drop(columns = ['Location'])

df

### Prediction Imputation Methods for filling missing values

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.utils import to_categorical

#Convert categorical data to numerical — complete this task for non-missing value dataset.
oneHotEncoder = OneHotEncoder(sparse = False)
labelEncoder = LabelEncoder()

integerEncoded = labelEncoder.fit_transform(dfNoMissing["WindGustDir"])
oneHotEncoded = to_categorical(integerEncoded)
print(oneHotEncoded)
print(pd.Series(np.array(oneHotEncoded)))
print(dfNoMissing["WindGustDir"].to_numpy())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


Exception: ignored

In [ ]:
print(dfNoMissing["WindGustDir"])

6049      0.0
6050      0.0
6052      0.0
6053      0.0
6054      0.0
         ... 
142298    0.0
142299    1.0
142300    1.0
142301    0.0
142302    0.0
Name: WindGustDir, Length: 56420, dtype: float32


In [ ]:
dfNoMissing

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,0.0,48.0,ENE,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,0.0,37.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,0.0,46.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,0.0,31.0,WNW,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,0.0,35.0,NW,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,2017-06-20,Darwin,19.3,33.4,0.0,6.0,11.0,0.0,35.0,SE,NE,9.0,20.0,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,No,No
142299,2017-06-21,Darwin,21.2,32.6,0.0,7.6,8.6,1.0,37.0,SE,SE,13.0,11.0,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,No,No
142300,2017-06-22,Darwin,20.7,32.8,0.0,5.6,11.0,1.0,33.0,E,W,17.0,11.0,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,No,No
142301,2017-06-23,Darwin,19.5,31.8,0.0,6.2,10.6,0.0,26.0,SE,NNW,9.0,17.0,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,No,No


In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

data = data[["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]]

data["Sex"] = [1 if x=="male" else 0 for x in data["Sex"]]

test_data = data[data["Age"].isnull()]
data.dropna(inplace=True)

y_train = data["Age"]
X_train = data.drop("Age", axis=1)
X_test = test_data.drop("Age", axis=1)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
df.head()

In [ ]:
#Dataset is adjusted with z-score normalization (i.e. standardization) because it handles outliers well.

from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(df)
df = scaler.transform(df)
df